<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# Custom Modules Management

This tutorial aims to guide you in the management of your custom modules and in the resolution of a simple problem using FAST-OAD. 
The problem under consideration consists in a cantilever beam of length **L** with a rectangular section of height **h** and width **l**. A force **F** is applied at the tip of the beam that also undergoes its proper weight homogeneously distributed **w**. 

The objectives of this tutorial will be to create and call custom modules through FAST-OAD to solve the following problems:
* For **F** = 1000N, **L** = 2.0m, **l** = 0.3m: Find the height **h** that leads to a displacement of 0.02m considering an aluminium beam. 
* For **F** = 1e5N, **L** = 10.0m, **l** = 0.3m: Find the height **h** so that the maximum normal stress within the beam does not exceed the aluminium yield stress (450e6 Pa).

<div class="row">
  <div class="column">
    <img src="./img/problem_description.png" width="600">
  </div>
</div>

## Imports

In [ ]:
import os.path as pth
import logging
import fastoad.api as oad

## Directories and files definition 

In [ ]:
DATA_FOLDER = "data"

WORK_FOLDER = "workdir"

CONFIGURATION_FILE_NAME = pth.join(DATA_FOLDER, "beam_problem.yml")

logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

## 1. Custom Modules

The following modules are created: [section_properties.py](./modules/section_properties.py), [weight.py](./modules/weight.py), [displacements.py](./modules/displacements.py), [stresses.py](./modules/stresses.py). They respectively compute: 
* The height of the beam for a given moment of inertia and width. 
* The linear weight **w** from material density and section dimensions. 
* The necessary moment of inertia to reach a given displacement. 
* The necessary moment of inertia not to overcome the material yield stress. 

As you may notice, each module is registered using the decorator `@oad.RegisterOpenMDAOSystem` and a unique id. 

An example is provided for the geometry module that computes the height of the beam for a given moment of inertia and a given width: 

``` python
import numpy as np
import openmdao.api as om

import fastoad.api as oad


@oad.RegisterOpenMDAOSystem("fastoad.beam_problem.geometry")
class RectangularSection(om.ExplicitComponent):
    def setup(self):

        self.add_input("data:beam_problem:geometry:l", val=np.nan, desc="Section width", units="m")
        self.add_input(
            "data:beam_problem:geometry:Ixx",
            val=np.nan,
            desc="Section second moment of area along w.r.t. x axis",
            units="m**4",
        )
        self.add_output("data:beam_problem:geometry:h", val=0.01, desc="Section height", units="m")

        self.declare_partials("*", "*", method="fd")

    def compute(self, inputs, outputs):
        l = inputs["data:beam_problem:geometry:l"]
        I_xx = inputs["data:beam_problem:geometry:Ixx"]

        outputs["data:beam_problem:geometry:h"] = (12 * I_xx / l) ** (1 / 3)
```

## 2. Configuration file
Once the modules are created, the problem can be defined through the [configuration file](./data/beam_problem.yml): 
* The folder(s) containing your custom modules is (are) indicated within the `module_folders` section.
* Your problem is defined within the `model` section. 
* The order of the modules in the `model` section will drive the order of execution. If at least one output variable of a module is an input of a previous one, a loop occurs and and `nonlinear_solver` and `linear_solver` must be defined. Here non linear Block Gauss-Seidel and linear Direct solvers are used. 

The configuration file for the first problem is shown below: 

```yaml
title: Sample OAD Process

# List of folder paths where user added custom registered OpenMDAO components
module_folders:
    - ../modules

# Input and output files
input_file: ./problem_inputs.xml
output_file: ../workdir/problem_outputs.xml

# Definition of problem driver assuming the OpenMDAO convention "import openmdao.api as om"
driver: om.ScipyOptimizeDriver(tol=1e-2, optimizer='COBYLA')

# Definition of OpenMDAO model
# Although "model" is a mandatory name for the top level of the model, its
# sub-components can be freely named by user
model:

  #  Solvers are defined assuming the OpenMDAO convention "import openmdao.api as om"
  nonlinear_solver: om.NonlinearBlockGS(maxiter=100, atol=1e-2, iprint=1)
  linear_solver: om.DirectSolver()
  
  geometry:
    id: "fastoad.beam_problem.geometry"
    
  weight:
    id: "fastoad.beam_problem.weight"
    
  displacements:
    id: "fastoad.beam_problem.disp"
```

As you can notice, three different modules are called sequentially: `geometry`, `weight` and `displacements`. 

As the variable `data:beam_problem:geometry:Ixx` is an output of the displacements module and an input of the geometry one, a loop is created and we need to introduce solvers.

This loop can be easily observed by plotting the $N^2$ diagram that allows you to check the connections between your models:

In [ ]:
N2_FILE = pth.join(WORK_FOLDER, 'n2.html')
oad.write_n2(CONFIGURATION_FILE_NAME, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width='100%', height='500px')

## 3. Input file 

Once the configuration file is generated you can generate or use an already generated input file: 

In [ ]:
# UNCOMMENT THE FOLLOWING LINE TO GENERATE A BLANK INPUT FILE 
#oad.generate_inputs(CONFIGURATION_FILE_NAME, overwrite=True)
INPUT_FILE_NAME = pth.join(DATA_FOLDER, "problem_inputs.xml")
oad.variable_viewer(INPUT_FILE_NAME)

## 4. Run the problem 

You can now run the problem:

In [ ]:
disp_problem = oad.evaluate_problem(CONFIGURATION_FILE_NAME, overwrite=True)

In [ ]:
oad.variable_viewer(disp_problem.output_file_path)

## 5. Change models 

To solve the second problem -- that consists in finding the beam height necessary to sustain the external loads without exceeding the material yield stress -- you have to replace the last module. Here again everything happens in the [configuration file](./data/beam_problem_stress.yml).

You have to replace the ```id: ``` of the last module: "fastoad.beam_problem.disp" by the new one "fastoad.beam_problem.stresses". 

You can also update the name of the module, replacing "displacements" by "stresses" as illustrated in the example. This is not a mandatory action but it should be used to keep the readability, especially if the action perfomed by the module changes. 

<u>*Note:*</u> *Save the configuration file with a different name to run the two problems independently.*

<div class="row">
  <div class="column">
    <img src="./img/stress_config.gif" width="800">
  </div>
</div>

Then generate the input file and update the input data if necessary. 

In [ ]:
CONFIGURATION_FILE_STRESS_NAME = pth.join(DATA_FOLDER, "beam_problem_stress.yml")
# UNCOMMENT THE FOLLOWING LINE TO GENERATE A BLANK INPUT FILE 
#oad.generate_inputs(CONFIGURATION_FILE_STRESS_NAME, overwrite=True)
INPUT_FILE_NAME = pth.join(DATA_FOLDER, "problem_inputs_stress.xml")
oad.variable_viewer(INPUT_FILE_NAME)

In [ ]:
stress_problem = oad.evaluate_problem(CONFIGURATION_FILE_STRESS_NAME, overwrite=True)

In [ ]:
oad.variable_viewer(stress_problem.output_file_path)

# Variable description file

You can complete your models with a detailed description of your variables. This action is of particular interest if you share them. 

To do so, in the folder(s) containing your modules you have to create a text file with the name `variable_descriptions.txt`. Then, fill this file with the variables and their description as follows:   

```text
# Custom modules variables description file
data:beam_problem:geometry:l || Beam section width
data:beam_problem:geometry:h || Beam section height
data:beam_problem:geometry:L || Beam length
data:beam_problem:geometry:Ixx || Beam section second moment of area w.r.t. x axis 
data:beam_problem:material:density || Material density 
data:beam_problem:material:yield_stress || Material yield stress
data:beam_problem:material:E || Material Young's modulus
data:beam_problem:weight:linear_weight || Beam linear weight
data:beam_problem:displacements:target || Beam targeted displacement when loads are applied
data:beam_problem:forces:F || Force applied at the tip of the beam

```

Those descriptions are now printed in the dedicated column when the `variable_viewer` is called.